In [1]:
search_for = 138
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.1354532241821289
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 1097396311
type: [1, 1, 1, 1, 138] 138
cases of this type: 362673936
100000 60.88708399388145
200000 104.52259475109527
300000 104.3289142710335
400000 105.34213375558954
500000 101.58595036636066
600000 102.04553079385825
700000 106.398852890254
800000 102.01339070533791
900000 100.86437115456656
1000000 100.33334285890173
1100000 100.69048015809119
1200000 100.49470580443848
1300000 82.19517021734903
1400000 40.96614999038298
1500000 99.71672047106225
1600000 99.41117967327659
1700000 99.98372560310736
1800000 98.78971329331367
1900000 100.34228500206235
2000000 103.60628577744029
2100000 101.75368741064553
2200000 98.31183926293554
2300000 100.49245482112737
2400000 99.97494507256981
2500000 100.97254631869424
2600000 98.47926275857189
2700000 24.416515914598005
2800000 96.89558835824481
2900000 99.59914687545538
3000000 97.95

29400000 111.5218889331897
29500000 111.09873770582958
29600000 108.55701352944827
29700000 111.95321499694708
29800000 109.50766016361393
29900000 108.97247109039387
30000000 106.14869011221649
30100000 105.32541724848925
30200000 91.14195382683431
30300000 35.6640532028373
30400000 101.78925353357798
30500000 107.2629311313136
30600000 105.03166527985974
30700000 110.83042714947118
30800000 104.71532197186116
30900000 105.89564098958603
31000000 111.38183535130212
31100000 111.89507619626036
31200000 115.65280731413186
31300000 104.8779891313034
31400000 44.51462955573838
31500000 113.06783001894495
31600000 110.61134744893877
31700000 106.90900957212261
31800000 50.004583625318055
31900000 90.21573418645495
32000000 107.33095848120624
32100000 105.65685046582244
32200000 105.35287052112318
32300000 103.47922764403934
32400000 101.00495482697475
32500000 104.44362456069702
32600000 101.15768062382536
32700000 101.24883850046538
32800000 99.40376164311964
32900000 24.69800215095131
33

59200000 23.216706016927656
59300000 97.79723381326833
59400000 95.74627741601306
59500000 97.25209443909536
59600000 99.08486345917379
59700000 99.5138451762292
59800000 99.14229385007609
59900000 101.47771087940609
60000000 103.28102955253266
60100000 43.872488304194064
60200000 105.43471460745975
60300000 103.48466776213435
60400000 104.98960842593908
60500000 107.24582933357468
60600000 111.46043273050252
60700000 108.84147023425453
60800000 107.05882179348066
60900000 67.47281498139779
61000000 78.73156225301616
61100000 108.60123833022587
61200000 103.07873361144729
61300000 103.73161734199253
61400000 112.53013119125255
61500000 99.24253670385903
61600000 97.45364428267261
61700000 96.45595285951043
61800000 33.897421658950556
61900000 89.09713113731868
62000000 99.5261367323431
62100000 97.81280016025579
62200000 105.06246598636909
62300000 101.22341341731448
62400000 102.68028207902903
62500000 105.27462930832696
62600000 106.58536920585809
62700000 36.64152673715965
62800000 

89200000 109.11178859654315
89300000 114.07023684084712
89400000 110.87476741492064
89500000 112.17551610895607
89600000 107.12117079804268
89700000 100.54363462182116
89800000 106.94801040344399
89900000 105.05602870260383
90000000 73.86660882886383
90100000 53.80215853195343
90200000 96.42621834964407
90300000 90.67298137711197
90400000 93.3985373382279
90500000 98.64920943324555
90600000 88.64982328401193
90700000 38.5864475631906
90800000 83.24722773200516
90900000 92.50478890091502
91000000 96.44058115201406
91100000 94.7702006625181
91200000 91.45717657254515
91300000 95.76020177224089
91400000 33.82854079889282
91500000 101.23276811086252
91600000 100.3592224419762
91700000 106.21359862597158
91800000 98.6747068970945
91900000 107.81125896275732
92000000 115.36788284685576
92100000 104.81978221842719
92200000 110.43847886142684
92300000 107.1187666737285
92400000 104.53609367046877
92500000 106.62329697678862
92600000 101.4188323510235
92700000 36.9577230923938
92800000 91.52183

118600000 105.43799761266393
118700000 96.40321039041895
118800000 107.89153056736697
118900000 91.59058133000579
119000000 95.77477188117176
119100000 85.9240816140342
119200000 40.974689574044426
119300000 87.84062326079646
119400000 97.65089166349895
119500000 93.24725814494873
119600000 48.46024682902367
119700000 68.69158522151429
119800000 96.11019285540713
119900000 89.19528481104336
120000000 96.03878093761523
120100000 33.16115539219279
120200000 98.35253380615623
120300000 90.27370248780427
120400000 98.38625474702381
120500000 99.00622087461193
120600000 98.65353674751664
120700000 108.70498589359613
120800000 98.75799530130075
120900000 113.02080042645323
121000000 99.71042750812214
121100000 94.09199994809521
121200000 103.20733185110797
121300000 99.89077991738523
121400000 91.76556065072121
121500000 96.7007885854582
121600000 86.37929994930995
121700000 91.55142003597828
121800000 43.090392518969736
121900000 75.82748218892942
122000000 89.18704638072097
122100000 91.81

...

983100000 7.009601080988141
983200000 9.77976541552173
983300000 10.80449334534753
983400000 8.789270057786192
983500000 7.804568175434996
983600000 6.959484635147272
983700000 1.8991256176372642
983800000 7.491663201214635
983900000 7.739997505977407
984000000 8.24982541595641
984100000 7.140150987230825
984200000 6.813930765339029
984300000 3.9258172014073964
984400000 3.4997602798925898
984500000 7.182517335407313
984600000 6.901514190269251
984700000 8.17388411966894
984800000 7.057568920490001
984900000 6.181328327688677
985000000 3.4245101915991305
985100000 5.882627272363172
985200000 6.809173419725885
985300000 8.068559379158053
985400000 7.573514153391642
985500000 6.939595244282516
985600000 5.596725681842315
985700000 2.939346159309877
985800000 6.731236626271884
985900000 7.572444733645462
986000000 7.914488585133542
986100000 6.7359098294167445
986200000 5.992056195877442
986300000 3.873043392284512
986400000 5.493789866500981
986500000 7.265223564482536
986600000 7.513686

1011400000 2.1628672834846796
1011500000 1.8838899461531329
1011600000 2.216778569356453
1011700000 1.9968323932199616
1011800000 2.1835830828416505
1011900000 1.0191982804507347
1012000000 1.2076675617982149
1012100000 1.8694311206744763
1012200000 1.903168763910532
1012300000 2.0830842999768655
1012400000 1.6368493700355926
1012500000 1.4713446710265232
1012600000 0.4951404980865776
1012700000 1.664780693807647
1012800000 1.6784120502500641
1012900000 1.813485406906414
1013000000 1.5785010173947103
1013100000 1.4468533299317021
1013200000 0.8899098084631264
1013300000 0.6427308078477535
1013400000 1.5225968739745663
1013500000 1.4305187824537873
1013600000 1.648537072455099
1013700000 1.3468932854521534
1013800000 1.3196617815718055
1013900000 0.48726213460668655
1014000000 1.0239702159942443
1014100000 1.3065422483348348
1014200000 1.4775109408331142
1014300000 1.4440167440165605
1014400000 1.2733137759811084
1014500000 0.9472045259696721
1014600000 0.5269009599541552
1014700000 1.6

1037000000 1.558886423463554
1037100000 1.9526278820672736
1037200000 1.3035743917952836
1037300000 1.5598563194443442
1037400000 1.775569181864261
1037500000 1.4363845204109928
1037600000 1.5889817289812296
1037700000 1.7427231837847075
1037800000 1.8090601009509246
1037900000 1.6633130111264143
1038000000 1.6503552093796763
1038100000 1.4526642207147362
1038200000 1.605451947560233
1038300000 1.571280633541239
1038400000 1.7921194076654174
1038500000 1.6185499005918853
1038600000 1.6429052029745161
1038700000 1.379034954229007
1038800000 1.6216039423632482
1038900000 1.6850862425340052
1039000000 1.483336579354447
1039100000 1.48818864014811
1039200000 1.8459968425180993
1039300000 1.5384691453335089
1039400000 1.2873172010171525
1039500000 1.8996465167258971
1039600000 1.6857015081689417
1039700000 1.1608578117888657
1039800000 1.7657296492761956
1039900000 1.755694402271533
1040000000 1.4669175150845646
1040100000 1.8558654833620079
1040200000 2.1992383494389633
best so far: 0
type

1058100000 0.9412206870527227
1058200000 1.1137854668169498
1058300000 0.8245550853875411
1058400000 0.7622755938982705
1058500000 1.0505779903015315
1058600000 0.9127108736780662
1058700000 0.826386443481209
1058800000 0.9747468153282701
1058900000 0.9324874912451545
1059000000 0.8272578609061956
1059100000 0.899654040469857
1059200000 1.1078698976202712
1059300000 0.7224365122235696
1059400000 0.836732023498835
1059500000 1.08373350171568
1059600000 0.8461901921324816
1059700000 0.7098726411772073
1059800000 1.0079810635870543
1059900000 0.9771916501283937
1060000000 0.8024385229013562
1060100000 0.8654408708009628
1060200000 1.0252635879229017
1060300000 0.6566157826681376
1060400000 0.8450038236616884
1060500000 1.017829130778771
1060600000 0.771054402356112
1060700000 0.6917854435309204
1060800000 0.9912223878953305
1060900000 0.8302950450214107
1061000000 0.6685221540341456
1061100000 0.9826936523028453
1061200000 0.9580018223250071
1061300000 0.6322888639384024
1061400000 0.8009

1084600000 0.057959755167432625
1084700000 0.05768730108818544
best so far: 0
type: [2, 1, 3, 1, 23] 138
cases of this type: 2518569
1084800000 0.1348994374538415
1084900000 0.20625395923567416
1085000000 0.18304138787378468
1085100000 0.16807338979614378
1085200000 0.16532993184889755
1085300000 0.16577223500301108
1085400000 0.16765412341901859
1085500000 0.1681783516130785
1085600000 0.16760373944788906
1085700000 0.1755432457394388
1085800000 0.10471095247866512
1085900000 0.05939176377085712
1086000000 0.05896180053088996
1086100000 0.058518885225979485
1086200000 0.0578740900356551
1086300000 0.07442537640939818
1086400000 0.1854908560602367
1086500000 0.1584790238325735
1086600000 0.15179051176642908
1086700000 0.15003260799468157
1086800000 0.14564461008619267
1086900000 0.1438161279540956
1087000000 0.14103141073956488
1087100000 0.1443720153653701
1087200000 0.15556782068702182
best so far: 0
type: [2, 1, 3, 23, 1] 138
cases of this type: 109503
1087300000 0.13517854477098187